<a href="https://colab.research.google.com/github/DominikAlb/GryLiczbowe-python/blob/master/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

import calendar
import string
import time
from typing import List
from datetime import datetime, timedelta
import statistics as stat
import logging
import os
import random
import matplotlib.pyplot as plt


class Game:

    def __init__(self, name: string, debug: bool, min_val: int, max_val: int, randomInput: bool, *args):
        self.name: string = name
        self.debug = debug
        self.min_val = min_val
        self.max_val = max_val
        self.randomInput = randomInput
        self.games = []
        self.gameResults = []
        if len(args) == 1 and isinstance(args[0], int):
            self.n = args[0]
        elif len(args) == 2 and isinstance(args[0], int) and isinstance(args[1], int):
            self.n = args[0]
            self.m = args[1]
        elif len(args) == 4 and isinstance(args[0], int) and isinstance(args[1], int) and isinstance(args[2],
                                                                                                     int) and isinstance(
            args[3], int):
            self.n = args[0]
            self.min_val2 = args[1]
            self.max_val2 = args[2]
            self.m = args[3]

    def monteCarlo(self, numSpins: int) -> float:
        pass

    def lasVegas(self, numSpins: int):
        pass

    def statistics(self, arr: List[float]):
        mean = stat.mean(arr)
        var = stat.variance(arr)
        sd = stat.stdev(arr)

        print("Rezultat:\n")
        print("srednia, " + str(mean) + "\n")
        print("wariancja, " + str(var) + "\n")
        print("odchylenie standardowe, " + str(sd) + "\n")
        print("---------------------------------------------------------------\n")

        if self.debug:
            logging.info("Wariancja: " + str(var))
            logging.info("Odchylenie standardowe: " + str(sd))
            logging.info("srednia: " + str(mean))
            logging.info("Oczekiwany rezultat dla " + self.name + " to: " + str(round(100 * mean, 3)) + " % +/- " + str(
                round(100 * 1.96 * sd, 3)) + " % z 95% pewnoscia\n\n")

        return mean, var, sd

    def draft(self, min, max):
        #pass
        plt.ylabel("prawdopodobienstwo wygranej")
        plt.xlabel("liczba zakladow: " + self.name)
        plt.plot(self.games[min:max], self.gameResults[min:max])
        #if "LasVegas" in self.name:
        #    self.timeToWin()

    def show(self):
        #pass
        plt.show()

    @staticmethod
    def draw(min_val: int, max_val: int, numbers: List[int], numSpins: int) -> bool:
        outcome = random.sample(range(min_val, max_val), numSpins)
        # print("OUT: " + str(outcome) + " : " + str(numSpins))
        count: int = 0
        for x in outcome:
            if x in numbers:
                count = count + 1
        if count == numSpins:
            return True
        return False

    def saveLocally(self, text: string, folder: string) -> bool:
        if folder == '':
            folder = self.name
        try:
            directory = os.path.join("D:\\", "awss3\\", folder)
            with open(directory + "\\" + self.name + "-" + datetime.now().strftime("%H-%M-%S") + str(
                    self.randomInput) + ".csv",
                      'a+', encoding='UTF8') as f:
                f.write(text)
                f.close()

        except Exception as e:
            print(e)
            return False
        return True

    def loadLocally(self, folder: str) -> bool:

        try:
            fixdata = []
            directory = os.path.join("D:\\", "awss3\\", folder)
            for root, dirs, files in os.walk(directory):
                for file in files:
                    gameResults: string = ""
                    results = []
                    if file.endswith(".csv"):
                        f = open(os.path.join(directory + "\\" + file), 'r')
                        gameResults = f.read().split('\n')[0]
                        for s in gameResults.split(' '):
                            try:
                                results.append(float(s))
                            except Exception as e:
                                continue

                        self.gameResults = results
                        self.games = range(1, len(self.gameResults) + 1)
                        self.draft(0, len(self.gameResults))
                        f.close()
                    fixdata += results
                    del gameResults
                    del results
            mean, var, sd = self.statistics(fixdata)
            text: string = "\nsrednia: " + str(mean) + "\nwariancja: " + str(
                var) + "\nodchylenie standardowe: " + str(sd)
            #print(text)
            del fixdata
        except Exception as e:
            print(e)
            return False
        return True

    def isTemporaryData(self, isTemp: bool) -> string:
        tempFolder = ""
        if isTemp:
            tempFolder = "temp/"
        return tempFolder

    def timeToWin(self):

        drawPerWeek: int = 0
        drawPerDay: int = 0
        drawPerMinute: int = 0
        endDate = None

        if "EuroRoulette" in self.name:
            drawPerMinute = 2
        elif "Lotto" in self.name:
            drawPerDay = 1
            drawPerWeek = 2
        elif "MultiMulti" in self.name:
            drawPerDay = 2
            drawPerWeek = 7
        elif "EuroJackpot" in self.name:
            drawPerDay = 1
            drawPerWeek = 2
        elif "PowerBall" in self.name:
            drawPerDay = 1
            drawPerWeek = 2

        if drawPerMinute != 0:
            tempValue = self.gameResults[-1] * drawPerMinute
            endDate = timedelta(days=tempValue / 1440)
            _time = endDate.seconds / 1440
            return _time
        elif drawPerDay != 0 and drawPerWeek != 0:
            avgDrawPerDay = ((drawPerDay * drawPerWeek) / 7)
            if self.gameResults[-1] * 7 > timedelta.max.days:
                tempValue = self.gameResults[1] * avgDrawPerDay
            else:
                tempValue = self.gameResults[-1] / (drawPerDay * drawPerWeek) * avgDrawPerDay
            return tempValue


        #print("Czas do wygranej: ", endDate.year, " lat, ", endDate.month, " miesięcy, ", endDate.day, " dni, ", endDate.hour, " godzin, ", endDate.minute, " minut.")
        #print("Oczekiwany wynik okolo: ", datetime.now() + timedelta(days=endDate.year * 365 * endDate.month * 30 + endDate.day, hours=endDate.hour, minutes=endDate.minute))
        #print("Czas do wygranej: ", endDate.seconds / 1440, " dni, ")
        #print("Oczekiwany wynik okolo: ", datetime.now() + timedelta(days=_time))

        return -1

In [6]:
# This is a sample Python script.

# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

# Press the green button in the gutter to run the script.
from datetime import datetime
import logging
import time
import string
import statistics as stat


def play(length: int, game: Game, gameName: string):
    arr = []
    print(game.name)
    if gameName == "MonteCarlo":
        print("Monte Carlo")
        start_time = time.time()
        arr = [game.monteCarlo(3000) for _ in range(0, length)]
        print("--- %s minutes ---" % ((time.time() - start_time) / 60))
        game.show()
        mean, var, sd = game.statistics(arr)
        text: string = str(arr) + "\nsrednia: " + str(mean) + "\nwariancja: " + str(
            var) + "\nodchylenie standardowe: " + str(sd)
        print(text)
        #game.save(" ".join([str(i) for i in game.gameResults]), game.name, False)
    elif gameName == "LasVegas":
        print("Las Vegas")
        start_time = time.time()
        _tuple = [game.lasVegas(50) for _ in range(0, 100)]
        unzip = list(zip(*_tuple))
        arr, timeToWin = unzip[0], unzip[1]
        print("--- %s minutes ---" % ((time.time() - start_time)/60))
        game.show()
        temp = []
        for i in arr:
            #game.save(" ".join([str(j) for j in game.gameResults]), game.name, False)
            temp += i

        mean, var, sd = game.statistics(temp)
        text: string = "srednia: " + str(mean) + "\nwariancja: " + str(
                var) + "\nodchylenie standardowe: " + str(sd)
        print("sredni czas oczekiwania: ", stat.mean(timeToWin), " dni.")
        print(text)

if __name__ == '__main__':
    logging.info(datetime.now())
    length = 1000

    game: Game = EuroRoulette("EuroRoulette-MonteCarlo", True, 0, 37, True,  1)
    play(length, game, "MonteCarlo")
    game.show()

    game: Game = EuroRoulette("EuroRoulette-LasVegas", True, 0, 36, True, 1)
    play(length, game, "LasVegas")
    game.show()
    
    game = Lotto("Lotto-MonteCarlo", True, 1, 49, False, 6)
    play(length, game, "MonteCarlo")
    game.show()

    game = Lotto("Lotto-LasVegas", True, 1, 49, True, 6)
    play(length, game, "LasVegas")
    game.show()

    game = MultiMulti("MultiMulti-MonteCarlo", True, 1, 80, False, 20, 10)
    play(length, game, "MonteCarlo")
    game.show()

    game = MultiMulti("MultiMulti-LasVegas", True, 1, 80, True, 20, 10)
    play(length, game, "LasVegas")
    game.show()

    game = EuroJackpot("EuroJackpot-MonteCarlo", True, 1, 50, False, 5, 1, 10, 2)
    play(length, game, "MonteCarlo")
    game.show()

    game = EuroJackpot("EuroJackpot-LasVegas", True, 1, 50, True, 5, 1, 12, 2)
    play(length, game, "LasVegas")
    game.show()

    game = PowerBall("PowerBall-MonteCarlo", True, 1, 69, False, 5, 1, 26, 1)
    play(length, game, "MonteCarlo")
    game.show()

    game = PowerBall("PowerBall-LasVegas", True, 1, 69, True, 5, 1, 26, 1)
    play(length, game, "LasVegas")
    game.show()


NameError: ignored